In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-ed47  GCE       4                                       RUNNING  us-central1-a


In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [5]:
spark

In [6]:
bucket_name = '316431873' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

graphframes.sh
indexes/index2.pkl
indexses/index.pkl
indexses/index_anc.pkl
postings_gcp/0_000.bin
postings_gcp/0_001.bin
postings_gcp/0_002.bin
postings_gcp/0_003.bin
postings_gcp/0_004.bin
postings_gcp/0_005.bin
postings_gcp/0_006.bin
postings_gcp/0_007.bin
postings_gcp/0_008.bin
postings_gcp/0_009.bin
postings_gcp/0_010.bin
postings_gcp/0_011.bin
postings_gcp/0_012.bin
postings_gcp/0_013.bin
postings_gcp/0_014.bin
postings_gcp/0_015.bin
postings_gcp/0_016.bin
postings_gcp/0_017.bin
postings_gcp/0_018.bin
postings_gcp/0_019.bin
postings_gcp/0_020.bin
postings_gcp/0_021.bin
postings_gcp/0_022.bin
postings_gcp/0_023.bin
postings_gcp/0_024.bin
postings_gcp/0_025.bin
postings_gcp/0_026.bin
postings_gcp/0_027.bin
postings_gcp/0_028.bin
postings_gcp/0_posting_locs.pickle
postings_gcp/100_000.bin
postings_gcp/100_001.bin
postings_gcp/100_002.bin
postings_gcp/100_003.bin
postings_gcp/100_004.bin
postings_gcp/100_005.bin
postings_gcp/100_006.bin
postings_gcp/100_007.bin
postings_gcp/100_008.b

In [7]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)

In [8]:
# YOUR CODE HERE
def generate_graph(pages):
      edg = pages.flatMap(lambda tup: [Row(tup[0],data[0]) for data in tup[1] if data!="null"])
      return edg.distinct(), edg.flatMap(lambda tup: [Row(tup[0]),Row(tup[1])]).distinct()

In [9]:


pg_rnk_par = parquetFile.select("id", "anchor_text").rdd

# edges, vertices = generate_graph(pg_rnk_par)
# v_cnt, e_cnt = vertices.count(), edges.count()

In [10]:
edges, vertices = generate_graph(pg_rnk_par)


In [ ]:
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")


In [13]:
pr = pr.sort(col('pagerank').desc())


+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750|  9913.72878216078|
|  10568|  5385.34926364204|
|  32927| 5282.081575765276|
|  30680| 5128.233709604118|
|5843419| 4957.567686263869|
|  68253| 4769.278265355165|
|  31717|  4486.35018054831|
|  11867|  4146.41465091277|
|  14533|3996.4664408855015|
| 645042|3531.6270898037433|
|  17867|3246.0983906041424|
|5042916|2991.9457391661786|
|4689264|2982.3248830417474|
|  14532| 2934.746829203172|
|  25391|2903.5462235133987|
|   5405| 2891.416329154635|
|4764461| 2834.366987332662|
|  15573| 2783.865118158839|
|   9316| 2782.039646413771|
|8569916|2775.2861918400167|
+-------+------------------+
only showing top 20 rows



In [20]:
pr.repartition(1).write.csv(f'gs://{bucket_name}/pagerank/pr', compression="gzip")

In [19]:
pr_dict = pr_dict.collect()

22/01/09 11:32:49 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 1.0 in stage 325.0 (TID 4649) (cluster-ed47-w-1.us-central1-a.c.optimistic-jet-334418.internal executor 14): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_21904/602416628.py", line 6, in <lambda>
TypeError: transform() missing 1 required positional argument: 'f'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:517)
	at

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 325.0 failed 4 times, most recent failure: Lost task 1.3 in stage 325.0 (TID 4658) (cluster-ed47-w-1.us-central1-a.c.optimistic-jet-334418.internal executor 14): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_21904/602416628.py", line 6, in <lambda>
TypeError: tran